In [19]:
import random
import numpy as np
import matplotlib.pyplot as plt

In [95]:
def generateWord(length):
    abc = 'abcdefghijklmnopqrstuvwxyz'
    word = ''
    for i in range(length):
        word += random.choice(abc)
    return word

def generateLengths(count,mean, dist, showGraph = False):
    s = np.random.normal(mean, dist, count)
    s = np.round(s,0).astype(int)
    lessThanOne = s < 1
    s[lessThanOne] = 1
    if showGraph:
        count, bins, ignored = plt.hist(s, 30, normed=True)
        plt.show()
    return s

def generateDictionary(count):
    currentDict = []
    wordLengths = generateLengths(count,5,1)
    for length in wordLengths:
        oldLen = len(currentDict)
        while(len(currentDict) == oldLen):
            word = generateWord(length)
            if word not in currentDict:
                currentDict.append(word)  
    return currentDict

def generateComments(count,dictionary,mean,dist,showGraph = False):
    comments = []
    lengths = generateLengths(count,mean,dist,showGraph) 
    for length in lengths:
        comment = ''
        for i in range(length):
            comment += random.choice(dictionary) + ' '
        comments.append(comment.rstrip())
    return comments

In [96]:
generateWord(3)

'iws'

In [99]:
generateLengths(10,5,1)

array([6, 4, 5, 5, 5, 4, 6, 6, 5, 4])

In [100]:
d = generateDictionary(5)
d

['rvslki', 'qiiyd', 'ovglnk', 'hmb', 'twj']

In [101]:
print(generateComments(10,d,3,1,True))

['ovglnk qiiyd rvslki', 'rvslki qiiyd hmb', 'hmb rvslki twj', 'qiiyd qiiyd', 'twj rvslki hmb qiiyd', 'twj qiiyd ovglnk ovglnk', 'hmb hmb twj', 'rvslki qiiyd qiiyd', 'twj ovglnk qiiyd qiiyd', 'qiiyd rvslki ovglnk']
